### Importar librerías

In [ ]:
import os
import numpy as np
import datetime
import sklearn
from scipy import stats

# importar fichero con utilidades propias
from commons import myfunctions as myfunc


#### Definición de parámetros

Se indican parámetros necesarios en el notebook

In [ ]:
# parameters
SEMILLA = 42
B_TIPO="bin_s"
M_TIPO="mul_s"
DATA_TIPO="mul_s"
PRE_DATA_FILE=  "rows_transpose_norm_by_gene_id_with_target_num_"



Nombre del fichero con las muestras a utilizar

In [ ]:
DATA_FILE= PRE_DATA_FILE + DATA_TIPO

#### Incializar variables, comprobar entorno y leer el fichero con las muestras.

In [ ]:
start_time = datetime.datetime.now()

myfunc.reset_vars()

myfunc.NOTEBK_FILENAME = myfunc.get_nb_name()
myfunc.EXEC_DIR = os.path.join(myfunc.EXEC_DIR, M_TIPO)

myfunc.check_enviroment(myfunc.DATA_DIR, myfunc.CFDNA_DIR, myfunc.GENCODE_DIR, myfunc.H5_DIR, myfunc.LOG_DIR, myfunc.CSV_DIR, myfunc.MODEL_DIR, myfunc.EXEC_DIR, myfunc.MET_DIR)

def mutual_info_classif_state(X, y):
    return mutual_info_classif(X, y, random_state=SEMILLA)

#  leer fichero de datos
df_t = myfunc.read_h5_to_df(DATA_FILE, myfunc.H5_DIR)
display(df_t.groupby("target").size())
print("Shape df:",df_t.shape)


Se separan las muestras utilizando la misma semilla que se utilizó para el entrenamiento del modelo.

Se utilizan las muestras de _test_ para evaluar el rendimiento del modelo.

In [ ]:
from sklearn.model_selection import  train_test_split

X = df_t.iloc[:, :-1]
y = df_t.iloc[:, -1]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=True, random_state=42)

y_test_bin = y_test.copy()
y_test_bin[y_test_bin != 0] = 1

# y_train_bin = y_train.copy()
# y_train_bin[y_train_bin != 0] = 1

print("Shape X_test:",X_test.shape)

### Leer fichero con métricas calculadas

In [ ]:
archivo1= os.path.join(myfunc.MET_DIR, "metricas_0042.csv")

fichero1 = os.path.basename(archivo1)
carpeta1 = os.path.dirname(archivo1)

df_tmp = myfunc.read_csv_to_df_spa(fichero1, carpeta1)[['tipo', 'select', 'clasific', 'accuracy', 'precision', 'recall', 'f1_score', 'roc_auc', 'roc_auc_ovr','fichero_modelo','indices_auc','indices_jaccard']]
display(df_tmp.shape)


### Seleccion de modelos para clasificación binaria

Escoger los mejores para la clasificación binaria

In [ ]:
import pandas as pd
import ast

df1 = df_tmp[df_tmp.tipo.str.startswith(B_TIPO)].copy()
print(df1.columns)

display(df1.shape)
display(df1.sample(3)[["tipo","select","clasific","accuracy","roc_auc","fichero_modelo"]])


Buscar los 3 mejores modelos para clasificación binaria por auc.

In [ ]:
num_modelos_escoger = 3

df3 = df1.copy()
ficheros_modelos_bin_auc = []
ficheros_modelos_bin_auc1 =[]
for lista1 in df3.sort_values(by=["roc_auc","accuracy"], ascending=False)["fichero_modelo"].head(num_modelos_escoger):
    ficheros_modelos_bin_auc1.append(lista1)
ficheros_modelos_bin_auc.append(ficheros_modelos_bin_auc1)

print(ficheros_modelos_bin_auc)

### Seleccion de modelos para clasificación múltiple

Separar los indices de auc y de jaccard en columnas.

In [ ]:
import pandas as pd
import ast

df1 = df_tmp[df_tmp.tipo.str.startswith(M_TIPO)].copy()

# Usando apply junto con pd.Series para expandir los arrays en columnas separadas.
df1['auc_list'] = df1['indices_auc'].apply(lambda x: ast.literal_eval(x))
df1['jac_list'] = df1['indices_jaccard'].apply(lambda x: ast.literal_eval(x))

auc_expanded = df1['auc_list'].apply(pd.Series)
auc_expanded.columns = ['auc_0', 'auc_1', 'auc_2', 'auc_3', 'auc_4', 'auc_5', 'auc_6']
auc_expanded.reset_index(drop=True, inplace=True)

jac_expanded = df1['jac_list'].apply(pd.Series)
jac_expanded.columns = ['jac_0', 'jac_1', 'jac_2', 'jac_3', 'jac_4', 'jac_5', 'jac_6']
jac_expanded.reset_index(drop=True, inplace=True)

df1.reset_index(drop=True, inplace=True)

df2 = pd.concat([df1, auc_expanded], axis=1)

df2 = pd.concat([df2, jac_expanded], axis=1)

df2.drop(columns=['auc_list','roc_auc','jac_list'], inplace=True)

display(df2.shape)


Buscar los 3 mejores modelos por auc por cada tipo de cáncer.

In [ ]:
# Escogiendo los modelos por tipo
df3 = df2[df2.tipo==M_TIPO]

#  escoger el mejor modelos de cada clase pero de los diferentes algoritmos de seleccion
num_modelos_escoger_select = 3
ficheros_modelos_mul_auc = []
for clase1 in range(len(myfunc.TARGETS)):
    ficheros_modelos_mul_auc_tipo = []
    for lista1 in df3.sort_values(by=[f"auc_{clase1}","roc_auc_ovr"], ascending=False)["fichero_modelo"].head(num_modelos_escoger_select):
        ficheros_modelos_mul_auc_tipo.append(lista1)
    ficheros_modelos_mul_auc.append(ficheros_modelos_mul_auc_tipo)
display(ficheros_modelos_mul_auc)


#### Leer modelos guardados de clasificación binaria y multiclase

In [ ]:
ficheros_modelos_bin = ficheros_modelos_bin_auc
print(ficheros_modelos_bin_auc)

modelos_bin = []
for modelo_x in ficheros_modelos_bin:
    for modelo_x_1 in modelo_x:
        fichero1 = modelo_x_1
        modelos_bin.append(myfunc.read_modelo(myfunc.MODEL_DIR, fichero1))

ficheros_modelos_mul = ficheros_modelos_mul_auc
print(ficheros_modelos_mul_auc)

modelos_mul = []
for modelos_x in ficheros_modelos_mul:
    modelos = []
    for modelo_x_1 in modelos_x:        
        fichero1 = modelo_x_1
        modelos.append(myfunc.read_modelo(myfunc.MODEL_DIR, fichero1))
    modelos_mul.append(modelos)


### Fase 1. clasificación binaria

Se hace la predicción para cada modelo seleccionado y se queda con la moda, valores más frecuentes, de la predicción de cada muestra.


In [ ]:
# def consenso_bin(modelos_bin, X):
#   predicciones = []
#   for modelo1 in modelos_bin:
#     predict_bin = modelo1.predict(X)
#     predicciones.append(predict_bin)

#   # Convertir la lista de todas las predicciones en un array de numpy
#   todas_predicciones_array = np.array(predicciones)

#   # consenso_predicciones, _ = stats.mode(todas_predicciones_array, axis=0, keepdims=True)
#   consenso_predicciones, _ = stats.mode(todas_predicciones_array, axis=0)

#   return list(consenso_predicciones[0])

# predicciones_bin = consenso_bin(modelos_bin, X_test)

predicciones = []
for modelo1 in modelos_bin:
  predicciones.append(modelo1.predict(X_test))

# Convertir la lista de todas las predicciones en un array de numpy
todas_predicciones_array = np.array(predicciones)

# consenso_predicciones, _ = stats.mode(todas_predicciones_array, axis=0, keepdims=True)
consenso_predicciones, _ = stats.mode(todas_predicciones_array, axis=0)

predicciones_bin = list(consenso_predicciones[0])

print(sklearn.metrics.classification_report(y_test_bin, predicciones_bin))

conf_matrix = sklearn.metrics.confusion_matrix(y_test_bin, predicciones_bin)

accuracy = sklearn.metrics.accuracy_score(y_test_bin, predicciones_bin)
precision = sklearn.metrics.precision_score(y_test_bin, predicciones_bin, average="macro")
recall = sklearn.metrics.recall_score(y_test_bin, predicciones_bin, average="macro")
f1 = sklearn.metrics.f1_score(y_test_bin, predicciones_bin, average="macro")

print("\n")
print("           Exactitud: %.6f" % (accuracy)) 
print("   Precisión (media): %.6f" % (precision))
print("      Recall (media): %.6f" % (recall))
print("    F1-score (media): %.6f" % (f1))

myfunc.ver_metricas_bin_matriz_confusion(y_test_bin, predicciones_bin)


### Fase 2. clasificación multiclase


#### Haciendo un consenso por cada tipo


Tener en cuenta probabilidades ya calculadas para posteriores iteraciones

In [ ]:
# Inicializar el array para almacenar la clase de consenso y las probabilidades máximas asociadas para cada clase
consenso_clases = np.full(len(X_test), -1, dtype=int)  # Inicializar con -1 para indicar que no hay predicción todavía
consenso_proba_max_por_clase = np.zeros((len(X_test), len(modelos_mul)))  # Una columna para cada clase

# Almacenar todas las probabilidades de todos los modelos
probabilidades_todas = []

# Por cada tipo de cáncer se itera sobre los modelos escogidos que mejor predicen ese tipo
for i, conjunto_modelos in enumerate(modelos_mul):
    # Almacenar las probabilidades para este tipo de cáncer
    probabilidades_tipo = []
    probabilidades_proba_tipo = []
    
    # Obtener las probabilidades de cada modelo en el conjunto para cada muestra
    for modelo in conjunto_modelos:
        
        probabilidades1 = modelo.predict(X_test)
        probabilidades_tipo.append(probabilidades1)
        
        probabilidades_todas.append(probabilidades1)

        probabilidades_proba = modelo.predict_proba(X_test)
        probabilidades_proba_tipo.append(probabilidades_proba)
    
    # Calcular la media de las probabilidades para obtener la probabilidad de consenso 
    # consenso_predicciones, _ = stats.mode(probabilidades_tipo, axis=0, keepdims=True)
    consenso_predicciones, _ = stats.mode(probabilidades_tipo, axis=0)
    consenso_tipo = consenso_predicciones[0]

    consenso_proba_tipo = np.mean(probabilidades_proba_tipo, axis=0)[:,i]

    # Actualizar las clases de consenso y las probabilidades máximas solo para la clase actual (i)
    for j in range(len(X_test)):
        # Solo actualizamos si la nueva probabilidad es mayor que la máxima anterior para la que ha predicho como clase i
        if consenso_tipo[j] == i:
            if consenso_proba_tipo[j] > consenso_proba_max_por_clase[j, i]:
                consenso_proba_max_por_clase[j, i] = consenso_proba_tipo[j]
                consenso_clases[j] = i
    
# Con todas las probabilidades guardadas se calculan las modas por si queda alguna pendiente
# consenso_predicciones_todas, _ = stats.mode(probabilidades_todas, axis=0, keepdims=True)
consenso_predicciones_todas, _ = stats.mode(probabilidades_todas, axis=0)

# Si queda algurna pendiente, me quedo con la moda de todas las probabilidades guardadas
for j in range(len(X_test)):
    if consenso_clases[j] == -1:
        consenso_clases[j] = consenso_predicciones_todas[0][j]

predicciones_mul = list(consenso_clases)

#  Para el consenso final se conservan las prediciones del modelo de clasificación binaria
consenso_final = [predicciones_mul[i] if predicciones_bin[i] == 1 else 0 for i in range(len(predicciones_mul))]

y_pred = consenso_final.copy()

print(sklearn.metrics.classification_report(y_test, y_pred, zero_division=0, digits=3))

conf_matrix = sklearn.metrics.confusion_matrix(y_test, y_pred)

accuracy = sklearn.metrics.accuracy_score(y_test, y_pred)
precision = sklearn.metrics.precision_score(y_test, y_pred, average="macro")
recall = sklearn.metrics.recall_score(y_test, y_pred, average="macro")
f1 = sklearn.metrics.f1_score(y_test, y_pred, average="macro")

print("\n")
print("           Exactitud: %.6f" % (accuracy)) 
print("   Precisión (media): %.6f" % (precision))
print("      Recall (media): %.6f" % (recall))
print("    F1-score (media): %.6f" % (f1))


myfunc.ver_metricas_multi_matriz_confusion(y_test,y_pred)



#### Finalización del notebook

In [ ]:
end_time = datetime.datetime.now()
total_time = end_time - start_time
myfunc.verbose(f"Notebook ha tardado {total_time} segundos")